# Get currencies live quotations

In [ ]:
# !pip install requests

In [2]:
# imports
from pathlib import Path
import pandas as pd
import requests

In [3]:
# api configurations
ccy_pair = 'EUR/CHF'
AUTH_HOST = "https://auth.getneo.com"
DATA_HOST = "https://data.getneo.com"
login_id = 'LOGIN'
api_key = 'PASSWORD'
bearer_token = None

# output file config
output_dir = Path("../data_output/")
output_file = f"{ccy_pair.replace('/', '')}.csv"

In [4]:
# Authenticate to get token
if not bearer_token:
    response = requests.get(f"{AUTH_HOST}/api/v1/auth/login/", params={"login_id": login_id, "api_key": api_key})
    if response.status_code != 200:
        raise PermissionError(f"Failed to authenticate as {login_id}")
    print(f"Authenticated as {login_id}")

    bearer_token = response.headers["Authorization"]

Authenticated as cashstory@getneo.com


In [5]:
# Get data with token
response = requests.get(f"{DATA_HOST}/api/v1/prices/{ccy_pair}", headers={"Authorization": bearer_token})
if response.status_code != 200:
    bearer_token = None
    raise ValueError(f"Failed to retrieve {ccy_pair} prices")

In [6]:
# Result
prices_dict = response.json()
prices = pd.DataFrame(data=list(prices_dict.items()), columns=["DATE", "VALUE"])
oldest_date = prices["DATE"].min()
print(f"Closing price for {ccy_pair} on {oldest_date}: {prices_dict[oldest_date]}")
newest_date = prices["DATE"].max()
print(f"Closing price for {ccy_pair} on {newest_date}: {prices_dict[newest_date]}")

Closing price for EUR/CHF on 2019-10-21: 1.0996
Closing price for EUR/CHF on 2019-11-19: 1.0971


In [7]:
# Saving output to file
prices.to_csv(output_dir / output_file, index=False)